## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto by
##                                             Hayford Tetteh
Segmenting and Clustering Neighborhoods in Toronto
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.
Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

### Importing all the required libraries

In [3]:
!pip install beautifulsoup4
!pip install lxml
!pip install geopy
!pip install folium
import folium # 
!conda install -c conda-forge folium=0.5.0 --yes
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
import matplotlib.colors as colors
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [4]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


### 1. Downloading the Data From Wikipedia
This is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city
of Toronto in the province of Ontario. Only the first three characters are listed, corresponding to the Forward Sortation Area.

In [7]:
Toronto_Can_df = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(Toronto_Can_df, 'lxml')

print('soup ready')

soup ready


In [8]:
print(soup.title.text)

List of postal codes of Canada: M - Wikipedia


In [9]:
soup('a')

[<a id="top"></a>,
 <a class="mw-jump-link" href="#mw-head">Jump to navigation</a>,
 <a class="mw-jump-link" href="#p-search">Jump to search</a>,
 <a href="/wiki/Postal_codes_in_Canada" title="Postal codes in Canada">postal codes in Canada</a>,
 <a href="/wiki/Toronto" title="Toronto">Toronto</a>,
 <a href="/wiki/Ontario" title="Ontario">Ontario</a>,
 <a href="/wiki/Canada_Post" title="Canada Post">Canada Post</a>,
 <a href="#cite_note-1">[1]</a>,
 <a href="/wiki/Mobile_app" title="Mobile app">applications</a>,
 <a class="mw-redirect" href="/wiki/Smartphones" title="Smartphones">smartphones</a>,
 <a href="/wiki/IPhone" title="IPhone">iPhone</a>,
 <a href="/wiki/BlackBerry" title="BlackBerry">BlackBerry</a>,
 <a href="#cite_note-2">[2]</a>,
 <a href="/wiki/CD-ROM" title="CD-ROM">CD-ROMs</a>,
 <a href="/wiki/Toronto" title="Toronto">Toronto</a>,
 <a href="/wiki/Postal_codes_in_Canada#Forward_sortation_areas" title="Postal codes in Canada">FSAs</a>,
 <a href="/w/index.php?title=List_of_po

In [10]:
for link in soup.find_all("a"):
    print("Inner Text: {}".format(link.text))
    print("Title: {}".format(link.get("title")))
    print("href: {}".format(link.get("href")))

Inner Text: 
Title: None
href: None
Inner Text: Jump to navigation
Title: None
href: #mw-head
Inner Text: Jump to search
Title: None
href: #p-search
Inner Text: postal codes in Canada
Title: Postal codes in Canada
href: /wiki/Postal_codes_in_Canada
Inner Text: Toronto
Title: Toronto
href: /wiki/Toronto
Inner Text: Ontario
Title: Ontario
href: /wiki/Ontario
Inner Text: Canada Post
Title: Canada Post
href: /wiki/Canada_Post
Inner Text: [1]
Title: None
href: #cite_note-1
Inner Text: applications
Title: Mobile app
href: /wiki/Mobile_app
Inner Text: smartphones
Title: Smartphones
href: /wiki/Smartphones
Inner Text: iPhone
Title: IPhone
href: /wiki/IPhone
Inner Text: BlackBerry
Title: BlackBerry
href: /wiki/BlackBerry
Inner Text: [2]
Title: None
href: #cite_note-2
Inner Text: CD-ROMs
Title: CD-ROM
href: /wiki/CD-ROM
Inner Text: Toronto
Title: Toronto
href: /wiki/Toronto
Inner Text: FSAs
Title: Postal codes in Canada
href: /wiki/Postal_codes_in_Canada#Forward_sortation_areas
Inner Text: edit


### 2. Processing-part-1: extracting raw table (from webpage)

Printing the head and tail, shape and info of Toronto_Can_df data frame

In [11]:
 Toronto_Can_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

Toronto_Can_df.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [12]:
Toronto_Can_df.tail(10)

,Postal code,Borough,Neighborhood
170,M9Y,Not assigned,NaN
171,M1Z,Not assigned,NaN
172,M2Z,Not assigned,NaN
173,M3Z,Not assigned,NaN
174,M4Z,Not assigned,NaN
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
179,M9Z,Not assigned,NaN


In [13]:
Toronto_Can_df.shape

(180, 3)

In [14]:
Toronto_Can_df.info

<bound method DataFrame.info of     Postal code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
7           M8A      Not assigned   
8           M9A         Etobicoke   
9           M1B       Scarborough   
10          M2B      Not assigned   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
15          M7B      Not assigned   
16          M8B      Not assigned   
17          M9B         Etobicoke   
18          M1C       Scarborough   
19          M2C      Not assigned   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
24          M7C      Not assigned   
25    

Removing 'Not Assigned' Values form the Data frame

In [15]:
# Dropping the rows where Borough is 'Not assigned'
Toronto_Can_df1 =  Toronto_Can_df[ Toronto_Can_df.Borough != 'Not assigned']

In [16]:
# Combining the neighbourhoods with same Postalcode
Toronto_Can_df2 = Toronto_Can_df1.groupby(['Postal code','Borough'], sort=False).agg(', '.join)
Toronto_Can_df2.reset_index(inplace=True)

In [17]:
# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
Toronto_Can_df2['Neighborhood'] = np.where(Toronto_Can_df2['Neighborhood'] == 'Not assigned',Toronto_Can_df2['Borough'], Toronto_Can_df2['Neighborhood'])


Printing Head And Tail Of the new cleaned Data Frame Toronto_Can_df2

In [18]:
Toronto_Can_df2.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [19]:
Toronto_Can_df2.tail(10)

,Postal code,Borough,Neighborhood
93,M8W,Etobicoke,Alderwood / Long Branch
94,M9W,Etobicoke,Northwest
95,M1X,Scarborough,Upper Rouge
96,M4X,Downtown Toronto,St. James Town / Cabbagetown
97,M5X,Downtown Toronto,First Canadian Place / Underground city
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...
102,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [20]:
Toronto_Can_df2.shape

(103, 3)

In [21]:
Toronto_Can_df2.info

<bound method DataFrame.info of     Postal code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25    

## 2. Add longitude and lattitute to Data Table
Downloading Coordinate from Geospatial_data(Latitude and Longitude)
and print the Head and Tail.

In [23]:
Toronto_Can_df3 = pd.read_csv("https://cocl.us/Geospatial_data") 
# Preview the first 5 lines of the loaded data
Toronto_Can_df3.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [25]:
Toronto_Can_df3.tail(10)

,Postal Code,Latitude,Longitude
93,M9A,43.667856,-79.532242
94,M9B,43.650943,-79.554724
95,M9C,43.643515,-79.577201
96,M9L,43.756303,-79.565963
97,M9M,43.724766,-79.532242
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


Merging The Coordinates and the CleanData Frame together

In [26]:
df_toronto_Can = pd.merge(Toronto_Can_df2, Toronto_Can_df3, how ='left', left_on = 'Postal code', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto_Can.drop("Postal Code", axis=1, inplace= False)
df_toronto_Can.head(10)

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,M5A,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,M7A,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353
7,M3B,North York,Don Mills,M3B,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,M4B,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937


In [27]:
df_toronto_Can.tail(10)

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
93,M8W,Etobicoke,Alderwood / Long Branch,M8W,43.602414,-79.543484
94,M9W,Etobicoke,Northwest,M9W,43.706748,-79.594054
95,M1X,Scarborough,Upper Rouge,M1X,43.836125,-79.205636
96,M4X,Downtown Toronto,St. James Town / Cabbagetown,M4X,43.667967,-79.367675
97,M5X,Downtown Toronto,First Canadian Place / Underground city,M5X,43.648429,-79.382280
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,M8Y,43.636258,-79.498509
102,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...,M8Z,43.628841,-79.520999


## 3 Clustering and Segmenting: create map of Toronto using latitude and longitude values

In [29]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [55]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
map_toronto

In [56]:
for lat, lng, borough, neighborhood in zip(
        df_toronto_Can['Latitude'], 
        df_toronto_Can['Longitude'], 
        df_toronto_Can['Borough'], 
        df_toronto_Can['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius= 5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

KeyError: 'Latitude'

In [42]:
CLIENT_ID = 'T1LUMOHYFVR21KSZSVKTKIFO1DPONRPOL4PGKBLWSYTPNDNV' # your Foursquare ID
CLIENT_SECRET = 'JBKZMRN5WS2PBXLZDUMMLCYJIDZELUNSSP3QKM51GSHXXWB0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 500
#print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)


Your credentails:
CLIENT_ID: T1LUMOHYFVR21KSZSVKTKIFO1DPONRPOL4PGKBLWSYTPNDNV
CLIENT_SECRET:JBKZMRN5WS2PBXLZDUMMLCYJIDZELUNSSP3QKM51GSHXXWB0


In [43]:
url

'https://api.foursquare.com/v2/venues/search?client_id=T1LUMOHYFVR21KSZSVKTKIFO1DPONRPOL4PGKBLWSYTPNDNV&client_secret=JBKZMRN5WS2PBXLZDUMMLCYJIDZELUNSSP3QKM51GSHXXWB0&ll=43.6534817,-79.3839347&v=20180604&radius=500&limit=100'

In [44]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [45]:
#Send the GET request and examine the results
results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df_toronto_Can = json_normalize(venues)


In [46]:
df_toronto_Can.head(10)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4c093ee0340720a153728493,NaN,CA,NaN,Canada,NaN,184,[Canada],"[{'label': 'display', 'lat': 43.65182710471462...",43.651827,-79.383949,NaN,NaN,City Hall Council Chambers,v-1586283594
1,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,4ad4c05ef964a5208ff620e3,100 Queen St. W.,CA,Toronto,Canada,at Bay St.,38,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,M5H 2N2,ON,Toronto City Hall,v-1586283594
2,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",False,5b193c42598e64002ca79b96,100 Queen St W,CA,Toronto,Canada,NaN,3,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.653454, 'lng':...",43.653454,-79.383952,M5H 2N2,ON,City of Toronto Civic Innovation Office,v-1586283594
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50885719498ea7b5aab3a74c,483 Bay St,CA,Toronto,Canada,NaN,130,"[483 Bay St, Toronto ON M5G 2C9, Canada]","[{'label': 'display', 'lat': 43.653436, 'lng':...",43.653436,-79.382314,M5G 2C9,ON,GoodLife Fitness Toronto Bell Trinity Centre,v-1586283594
4,"[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",False,4c0121fd9a950f47fa9208c6,100 Queen Street West,CA,Toronto,Canada,NaN,6,"[100 Queen Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65350358617817...",43.653504,-79.383866,NaN,ON,City Hall Podium Green Roof,v-1586283594
5,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",False,4c8938c8944e224b52e72285,100 Queen St W,CA,Toronto,Canada,City Hall,107,"[100 Queen St W (City Hall), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65263139770985...",43.652631,-79.383295,M5H 2N3,ON,Toronto Public Library,v-1586283594
6,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",False,5a81bb73e7a2370cf92452dc,NaN,CA,Toronto,Canada,NaN,47,"[Toronto ON M5G, Canada]","[{'label': 'display', 'lat': 43.653119, 'lng':...",43.653119,-79.384252,M5G,ON,Service Canada,v-1586283594
7,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4c41d47cd691c9b6fa4c8d0a,Queen street and belfair ave,CA,Toronto,Canada,NaN,66,"[Queen street and belfair ave, Toronto ON, Can...","[{'label': 'display', 'lat': 43.65322456230742...",43.653225,-79.383185,NaN,ON,Kew Gardens Playground,v-1586283594
8,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4b06ef4cf964a52072f322e3,100 Queen St W,CA,Toronto,Canada,NaN,105,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.65263298456729...",43.652633,-79.383361,M5H 2N2,ON,Cafe On The Square,v-1586283594
9,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",False,520e879c498ef161b728e25d,483 Bay St,CA,Toronto,Canada,NaN,15,"[483 Bay St, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.653567, 'lng':...",43.653567,-79.383787,NaN,ON,Gateway Newstands,v-1586283594


In [53]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,City Hall Council Chambers,City Hall,NaN,CA,NaN,Canada,NaN,184,[Canada],"[{'label': 'display', 'lat': 43.65182710471462...",43.651827,-79.383949,NaN,NaN,4c093ee0340720a153728493
1,Toronto City Hall,City Hall,100 Queen St. W.,CA,Toronto,Canada,at Bay St.,38,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,M5H 2N2,ON,4ad4c05ef964a5208ff620e3
2,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,CA,Toronto,Canada,NaN,3,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.653454, 'lng':...",43.653454,-79.383952,M5H 2N2,ON,5b193c42598e64002ca79b96
3,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,CA,Toronto,Canada,NaN,130,"[483 Bay St, Toronto ON M5G 2C9, Canada]","[{'label': 'display', 'lat': 43.653436, 'lng':...",43.653436,-79.382314,M5G 2C9,ON,50885719498ea7b5aab3a74c
4,Toronto Public Library,Library,100 Queen St W,CA,Toronto,Canada,City Hall,107,"[100 Queen St W (City Hall), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65263139770985...",43.652631,-79.383295,M5H 2N3,ON,4c8938c8944e224b52e72285


In [54]:
df_toronto_Can_filtered.head(10)

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,id
0,City Hall Council Chambers,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",NaN,CA,NaN,Canada,NaN,184,[Canada],"[{'label': 'display', 'lat': 43.65182710471462...",43.651827,-79.383949,NaN,NaN,4c093ee0340720a153728493
1,Toronto City Hall,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",100 Queen St. W.,CA,Toronto,Canada,at Bay St.,38,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...","[{'label': 'display', 'lat': 43.65313989695342...",43.653140,-79.383967,M5H 2N2,ON,4ad4c05ef964a5208ff620e3
2,City of Toronto Civic Innovation Office,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",100 Queen St W,CA,Toronto,Canada,NaN,3,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.653454, 'lng':...",43.653454,-79.383952,M5H 2N2,ON,5b193c42598e64002ca79b96
3,GoodLife Fitness Toronto Bell Trinity Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",483 Bay St,CA,Toronto,Canada,NaN,130,"[483 Bay St, Toronto ON M5G 2C9, Canada]","[{'label': 'display', 'lat': 43.653436, 'lng':...",43.653436,-79.382314,M5G 2C9,ON,50885719498ea7b5aab3a74c
4,Toronto Public Library,"[{'id': '4bf58dd8d48988d12f941735', 'name': 'L...",100 Queen St W,CA,Toronto,Canada,City Hall,107,"[100 Queen St W (City Hall), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65263139770985...",43.652631,-79.383295,M5H 2N3,ON,4c8938c8944e224b52e72285
5,City Hall Podium Green Roof,"[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",100 Queen Street West,CA,Toronto,Canada,NaN,6,"[100 Queen Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65350358617817...",43.653504,-79.383866,NaN,ON,4c0121fd9a950f47fa9208c6
6,Service Canada,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",NaN,CA,Toronto,Canada,NaN,47,"[Toronto ON M5G, Canada]","[{'label': 'display', 'lat': 43.653119, 'lng':...",43.653119,-79.384252,M5G,ON,5a81bb73e7a2370cf92452dc
7,Kew Gardens Playground,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",Queen street and belfair ave,CA,Toronto,Canada,NaN,66,"[Queen street and belfair ave, Toronto ON, Can...","[{'label': 'display', 'lat': 43.65322456230742...",43.653225,-79.383185,NaN,ON,4c41d47cd691c9b6fa4c8d0a
8,Cafe On The Square,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",100 Queen St W,CA,Toronto,Canada,NaN,105,"[100 Queen St W, Toronto ON M5H 2N2, Canada]","[{'label': 'display', 'lat': 43.65263298456729...",43.652633,-79.383361,M5H 2N2,ON,4b06ef4cf964a52072f322e3
9,Gateway Newstands,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",483 Bay St,CA,Toronto,Canada,NaN,15,"[483 Bay St, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.653567, 'lng':...",43.653567,-79.383787,NaN,ON,520e879c498ef161b728e25d
